# Create a forge and execute an evolution cycle
In this notebook, we will go through the very few steps needed to run a forge cycle for a given budget. 

As of today, the easiest way to experiment with Ebiose is to use the OpenAI API. To do so, all you have to do is to set your OpenAI API key via an .env file or by replacing `"your-open-api-key"` in the following code block:

In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

> 💡 Recall that you may need to add the root of the repository to your `PYTHONPATH` environment variable. You may also use a `.env` file to do so or execute the following:


In [2]:
import sys
sys.path.append("./../")


## Creating a basic forge

In ebiose, a **forge** is where custom agents are created to solve specific problems. The forge is the exclusive origin of new agents. Within each forge, architects agents orchestrate the creation and improvement of agents by reusing existing building blocks from the ecosystem.

To create a forge and run a cycle, you must provide the following:
- a description of the forge, which defines the problem that must be solved by generated agents;
- the expected format of the agent's input and output, defined as Pydantic models;
- an implementation of the `compute_fitness` abstract method that will be used by the forge to evaluate the generated agents.

Let's say we wich to generate agents specialized in solving math problems. The forge description could be:

In [3]:
forge_description = "Solving math word problems"

Next, we need to define the expected input and output formats of the generated agents. These formats are to be defined as Pydantic models. 

For instance, in our context of solving math problems, we want the agent input to be a string which will represent the math problem to be solved and the agent output to be composed of two fields:
- `solution` which will be the final solution to the math problem, given as an integer;
- `rationale` which will be the rationale behind the found solution.

The IO Pydantic models will thus be:

In [4]:
from pydantic import BaseModel

class AgentInput(BaseModel):
        math_problem: str

class AgentOutput(BaseModel):
    solution: int
    rationale: str

Lastly, we must provide a way of evaluating the generated agents through the implementation of the `compute_fitness` abstract method of `AgentForge` class. For the sake of demonstration, we will here return a random float between 0 and 1, so that we don't spend tokens at evaluation.

In [5]:
import random
random.seed(7)

from ebiose.core.agent import Agent
from ebiose.core.agent_forge import AgentForge

class BasicForge(AgentForge):
    async def compute_fitness(self, agent: Agent, compute_token_id: str, **kwargs: dict[str, any]) -> float:
        return random.random()

/Users/xabier/dev/ebiose-core/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


We can now instantiate the forge with the provided elements:

In [6]:
forge = BasicForge(
    name="Basic forge",
    description=forge_description,
    agent_input_model=AgentInput,
    agent_output_model=AgentOutput,
    default_generated_agent_engine_type="langgraph_engine",
    default_model_endpoint_id="azure-gpt-4o-mini"
)

# Running a forge cycle

Once the forge is instantiated, we can start generating agents by running a **forge cycle**. 

We must first define a list of available LLMs. Here, we will use GPT 4o mini only. 

In [7]:

# from ebiose.compute_intensive_batch_processor.compute_intensive_batch_processor import (
#     ComputeIntensiveBatchProcessor,
# )
# ComputeIntensiveBatchProcessor.initialize()

The forge cycle is based on an evolutionary algorithm that requires two types of specialized agents: 
- **architect agents** which are agents that generate other agents from scratch;
- **genetic operator agents** which generate agents from one or several other existing agents.

For now, hand-made architect and crossover agents are provided in the `GraphUtils` class. We can load them as follows:

In [8]:
# from ebiose.core.engines.graph_engine.utils import GraphUtils

# architect_agent = GraphUtils.get_architect_agent(model_endpoint_id="azure-gpt-4o-mini")
# crossover_agent = GraphUtils.get_crossover_agent(model_endpoint_id="azure-gpt-4o-mini")

These agents are required to instantiate an Ebiose ecosystem:

In [9]:
# from ebiose.core.ecosystem import Ecosystem

# ecosystem = Ecosystem(
#     initial_architect_agents=[architect_agent],
#     initial_genetic_operator_agents=[crossover_agent],
# )

The forge cycle can now be launched by providing the created ecosystem, a budget in dollars (the forge cycle will end once this budget is exhausted) and, optionally, a path in which created agents and fitness will be saved accross generations. Note that we need to use `asyncio.run` to launch the forge cycle.

> 🚨 Before executing the following cell, check the amount of budget you have allocated!

> 💡 If you are using VSCode, install the [*Markdown Preview Mermaid Support* extension](https://marketplace.visualstudio.com/items?itemName=bierner.markdown-mermaid) to allow the display of the generated agent's graphs.

In [10]:
import asyncio
import nest_asyncio

from ebiose.core.evo_forging_cycle import EvoForgingCylceConfig
nest_asyncio.apply()

from pathlib import Path
from datetime import UTC, datetime

# the path where results will be saved
current_time = datetime.now(UTC).strftime("%Y-%m-%d_%H-%M-%S")
SAVE_PATH = Path(f"./../data/") / current_time
if not SAVE_PATH.exists():
    SAVE_PATH.mkdir(parents=True)

# budget for the forge cycle, in dollars
BUDGET = 0.01

cycle_config = EvoForgingCylceConfig(
    budget=BUDGET,
    n_agents_in_population=2,
    n_selected_agents_from_ecosystem=0,
    replacement_ratio=0.5,
    save_path=SAVE_PATH
)

final_agents, final_fitness = asyncio.run(
    forge.run_new_cycle(config=cycle_config)
)


Starting a new cycle for forge Basic forge
****** Initializing agents population ******
Creating 2 new agents with architect agents...


  0%|          | 0/2 [00:00<?, ?it/s]
Initializing structured output agent for model AgentOutput (1)
100%|██████████| 2/2 [00:20<00:00, 10.26s/it]
Agent initialization cost: 0.0057550349999999995
Population initialized with 2 agents
Initialization of 2 agents took 0:00:20.541315
Budget left after initialization: 0.004244965000000001 $
****** Running generation 0 ******
Evaluating current population of 2 agents...
100%|██████████| 2/2 [00:00<00:00, 826.14it/s]
Agent agent-abee9a02-a5b8-4d75-84f9-16c005d2ff20 fitness: 0.45318437637077535, cost: 0.0
Agent agent-cf7039ea-969b-4a35-9b6b-7aa6cb19cb66 fitness: 0.29976699686368236, cost: 0.0
Evaluation took 0:00:00.006082 for a total cost of 0.0 $


# Agent ID: agent-abee9a02-a5b8-4d75-84f9-16c005d2ff20
## Fitness: 0.45318437637077535
```mermaid 
graph LR
	Start_Node[start_node] --> Math_Problem_Solver(MathProblemSolver)
	Math_Problem_Solver(MathProblemSolver) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative network of Large Language Models designed to solve math word problems through a structured and systematic approach. Each node in this network has a specific role: 

- The StartNode initiates the problem-solving process. 
- The LLM nodes analyze, reason, and generate solutions to the math word problems based on structured prompts, utilizing techniques like chain-of-thought reasoning and self-reflection. 
- The EndNode concludes the process by summarizing the findings.

Your goal is to contribute meaningfully to the problem-solving cycle by effectively processing the information, generating insights, and seamlessly passing the results to the next node in the network.
##### MathProblemSolver
Given the math word problem presented to you, follow these steps to generate a structured approach for solving it:

1. **Analysis**: Read the problem carefully and identify the key components, including the question, known variables, and any relationships between them.

2. **Breakdown**: Decompose the problem into manageable parts. What information is directly given? What needs to be calculated?

3. **Plan**: Develop a step-by-step plan to solve the problem. Clearly outline the mathematical operations you will use.

4. **Execution**: Perform the calculations required and derive the answer. 

5. **Self-Reflection**: After reaching a solution, verify the process:
   - Did you use the correct formulas and methods?
   - Is the answer reasonable given the context of the problem?
   - Are there alternative methods to reach the same solution?

6. **Communication**: Summarize your findings, including the answer to the problem and the methodology used. Ensure that the output is clear and concise for the next node in the network.

You are encouraged to think critically and creatively as you navigate through this problem. Your output will be crucial for the next node in the graph.

# Agent ID: agent-cf7039ea-969b-4a35-9b6b-7aa6cb19cb66
## Fitness: 0.29976699686368236
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node_1(Understanding Problem)
	Llm_Node_1(Understanding Problem) --> Llm_Node_2(Solving Problem)
	Llm_Node_2(Solving Problem) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are an AI model designed to solve math word problems through a collaborative network of Large Language Models (LLMs). Your task is to work within a structured graph consisting of distinct nodes that guide you through the problem-solving process. The nodes include an understanding phase, a solving phase, and an endpoint for delivering the final answer. Each node has a specific role:

1. **Understanding Problem (LLMNode 1)**: Your goal is to comprehend the math word problem and extract key information required for solving it, including identifying the mathematical operations involved.

2. **Solving Problem (LLMNode 2)**: Your task is to perform the necessary calculations and provide the solution based on the information received from the previous node.

Remember to communicate clearly and effectively to ensure a seamless flow of information between nodes, and reflect on your reasoning process where applicable.
##### Understanding Problem
Read the following math word problem carefully. Your task is to extract the key information needed to solve it. Identify any numerical values, relationships, and mathematical operations implied by the problem. After gathering this information, summarize what you have understood and specify the mathematical operations you believe are involved. Ensure you provide clear reasoning for your choices. Here is the problem: [Insert math word problem here].
##### Solving Problem
Based on the information and mathematical operations identified by the previous node, perform the necessary calculations to solve the problem. Ensure that you clearly outline each step in your reasoning, detailing how you arrived at the solution. If there are multiple steps involved, break them down methodically. After completing the calculations, present the final answer and reflect on whether this solution seems correct, offering any additional insights or clarifications if necessary.



Saving current state to ../data/2025-03-04_19-41-52/generation=0
Starting crossover and mutation...

Initializing structured output agent for model AgentOutput (2)
Number of offsprings: 1/1
Crossover and mutation completed in 0:00:14.073210 for a total cost of 0.00176121 $
Generation 0 completed in 0:00:14.103145 with a total cost of 0.00176121 $
Budget left after first generation: 0.0024837550000000007 $
****** Running generation 1 ******
Evaluating current population of 2 agents...
100%|██████████| 2/2 [00:00<00:00, 2167.04it/s]
Agent agent-cf7039ea-969b-4a35-9b6b-7aa6cb19cb66 fitness: 0.03920725704743766, cost: 0.0
Agent agent-e47992e7-babc-4ee7-b253-d3520b1de2c5 fitness: 0.6682158565343952, cost: 0.0
Evaluation took 0:00:00.013505 for a total cost of 0.0 $


# Agent ID: agent-e47992e7-babc-4ee7-b253-d3520b1de2c5
## Fitness: 0.6682158565343952
```mermaid 
graph LR
	Start_Node[start_node] --> Math_Problem_Solver(MathProblemSolver)
	Math_Problem_Solver(MathProblemSolver) --> Math_Problem_Analyst(MathProblemAnalyst)
	Math_Problem_Analyst(MathProblemAnalyst) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative network of Large Language Models designed to solve math word problems through a structured and systematic approach. Each node in this network has a specific role: \n\n- The StartNode initiates the problem-solving process. \n- The LLM nodes analyze, reason, and generate solutions to the math word problems based on structured prompts, utilizing techniques like chain-of-thought reasoning and self-reflection. \n- The EndNode concludes the process by summarizing the findings.\n\nYour goal is to contribute meaningfully to the problem-solving cycle by effectively processing the information, generating insights, and seamlessly passing the results to the next node in the network.
##### MathProblemSolver
Given the math word problem presented to you, follow these steps to generate a structured approach for solving it:\n\n1. **Analysis**: Identify the key components, including the question, known variables, and relationships.\n\n2. **Breakdown**: Decompose the problem into manageable parts. What is given? What needs calculation?\n\n3. **Plan**: Develop a step-by-step plan for solving the problem, detailing the mathematical operations required.\n\n4. **Execution**: Perform the required calculations to derive the answer.\n\n5. **Self-Reflection**: Verify the process:\n   - Did you use the correct formulas?\n   - Is the answer reasonable?\n   - Are there alternative methods?\n\n6. **Communication**: Summarize your findings, including the answer and methodology. Ensure clarity for the next node.
##### MathProblemAnalyst
Review the structured approach generated by the Math Problem Solver. Consider:\n1. Are there any logical flaws or missing steps?\n2. Is the proposed solution efficient and clear?\n3. Suggest alternative methods or simplifications if applicable.\n\nProvide a summary of your assessment and any recommendations for refining the solution.

# Agent ID: agent-cf7039ea-969b-4a35-9b6b-7aa6cb19cb66
## Fitness: 0.03920725704743766
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node_1(Understanding Problem)
	Llm_Node_1(Understanding Problem) --> Llm_Node_2(Solving Problem)
	Llm_Node_2(Solving Problem) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are an AI model designed to solve math word problems through a collaborative network of Large Language Models (LLMs). Your task is to work within a structured graph consisting of distinct nodes that guide you through the problem-solving process. The nodes include an understanding phase, a solving phase, and an endpoint for delivering the final answer. Each node has a specific role:

1. **Understanding Problem (LLMNode 1)**: Your goal is to comprehend the math word problem and extract key information required for solving it, including identifying the mathematical operations involved.

2. **Solving Problem (LLMNode 2)**: Your task is to perform the necessary calculations and provide the solution based on the information received from the previous node.

Remember to communicate clearly and effectively to ensure a seamless flow of information between nodes, and reflect on your reasoning process where applicable.
##### Understanding Problem
Read the following math word problem carefully. Your task is to extract the key information needed to solve it. Identify any numerical values, relationships, and mathematical operations implied by the problem. After gathering this information, summarize what you have understood and specify the mathematical operations you believe are involved. Ensure you provide clear reasoning for your choices. Here is the problem: [Insert math word problem here].
##### Solving Problem
Based on the information and mathematical operations identified by the previous node, perform the necessary calculations to solve the problem. Ensure that you clearly outline each step in your reasoning, detailing how you arrived at the solution. If there are multiple steps involved, break them down methodically. After completing the calculations, present the final answer and reflect on whether this solution seems correct, offering any additional insights or clarifications if necessary.



Saving current state to ../data/2025-03-04_19-41-52/generation=1
Starting crossover and mutation...
Number of offsprings: 1/1
Crossover and mutation completed in 0:00:16.400448 for a total cost of 0.00200541 $
Generation 1 completed in 0:00:16.447155 with a total cost of 0.00200541 $
Budget left after new generation: 0.0004783449999999998 $
100%|██████████| 2/2 [00:00<00:00, 2477.44it/s]
Agent agent-e47992e7-babc-4ee7-b253-d3520b1de2c5 fitness: 0.9930959394666341, cost: 0.0
Agent agent-6e32241e-d0a5-4731-a3c6-8e63b63973db fitness: 0.8219247866097149, cost: 0.0
Saving current state to ../data/2025-03-04_19-41-52/generation=2
Cycle completed in 0:00:51.127876 with a total cost of 0.009521655 $
Budget left at final: 0.0004783449999999998 $
Returning 10 best agents


We can now display the best agents that have been returned as follows. Note that:
- all agents can be found in the `SAVE_PATH` directory if you defined one;
- here, the compute fitness only returns a random float, so the following displayed agents have not been truly evaluated. 

Go check [examples/math_forge/math_forge.py](./../examples/math_forge/math_forge.py) to see a fully implemeted forge with a non-random fitness evaluation function.

In [11]:
forge.display_results(final_agents, final_fitness)

# Agent ID: agent-e47992e7-babc-4ee7-b253-d3520b1de2c5
## Fitness: 0.9930959394666341
```mermaid 
graph LR
	Start_Node[start_node] --> Math_Problem_Solver(MathProblemSolver)
	Math_Problem_Solver(MathProblemSolver) --> Math_Problem_Analyst(MathProblemAnalyst)
	Math_Problem_Analyst(MathProblemAnalyst) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative network of Large Language Models designed to solve math word problems through a structured and systematic approach. Each node in this network has a specific role: \n\n- The StartNode initiates the problem-solving process. \n- The LLM nodes analyze, reason, and generate solutions to the math word problems based on structured prompts, utilizing techniques like chain-of-thought reasoning and self-reflection. \n- The EndNode concludes the process by summarizing the findings.\n\nYour goal is to contribute meaningfully to the problem-solving cycle by effectively processing the information, generating insights, and seamlessly passing the results to the next node in the network.
##### MathProblemSolver
Given the math word problem presented to you, follow these steps to generate a structured approach for solving it:\n\n1. **Analysis**: Identify the key components, including the question, known variables, and relationships.\n\n2. **Breakdown**: Decompose the problem into manageable parts. What is given? What needs calculation?\n\n3. **Plan**: Develop a step-by-step plan for solving the problem, detailing the mathematical operations required.\n\n4. **Execution**: Perform the required calculations to derive the answer.\n\n5. **Self-Reflection**: Verify the process:\n   - Did you use the correct formulas?\n   - Is the answer reasonable?\n   - Are there alternative methods?\n\n6. **Communication**: Summarize your findings, including the answer and methodology. Ensure clarity for the next node.
##### MathProblemAnalyst
Review the structured approach generated by the Math Problem Solver. Consider:\n1. Are there any logical flaws or missing steps?\n2. Is the proposed solution efficient and clear?\n3. Suggest alternative methods or simplifications if applicable.\n\nProvide a summary of your assessment and any recommendations for refining the solution.

# Agent ID: agent-6e32241e-d0a5-4731-a3c6-8e63b63973db
## Fitness: 0.8219247866097149
```mermaid 
graph LR
	Start_Node[start_node] --> Math_Problem_Solver(MathProblemSolver)
	Math_Problem_Solver(MathProblemSolver) --> Math_Problem_Analyst(MathProblemAnalyst)
	Math_Problem_Analyst(MathProblemAnalyst) --> Math_Problem_Reflector(MathProblemReflector)
	Math_Problem_Reflector(MathProblemReflector) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative network of Large Language Models designed to solve math word problems through a structured and systematic approach. Each node in this network has a specific role: \\n\\n- The StartNode initiates the problem-solving process. \\n- The LLM nodes analyze, reason, and generate solutions to the math word problems based on structured prompts, utilizing techniques like chain-of-thought reasoning and self-reflection. \\n- The EndNode concludes the process by summarizing the findings.\\n\\nYour goal is to contribute meaningfully to the problem-solving cycle by effectively processing the information, generating insights, and seamlessly passing the results to the next node in the network.
##### MathProblemSolver
Given the math word problem presented to you, follow these steps to generate a structured approach for solving it:\\n\\n1. **Analysis**: Identify the key components, including the question, known variables, and relationships.\\n\\n2. **Breakdown**: Decompose the problem into manageable parts. What is given? What needs calculation?\\n\\n3. **Plan**: Develop a step-by-step plan for solving the problem, detailing the mathematical operations required.\\n\\n4. **Execution**: Perform the required calculations to derive the answer.\\n\\n5. **Self-Reflection**: Verify the process:\\n   - Did you use the correct formulas?\\n   - Is the answer reasonable?\\n   - Are there alternative methods?\\n\\n6. **Communication**: Summarize your findings, including the answer and methodology. Ensure clarity for the next node.
##### MathProblemAnalyst
Review the structured approach generated by the Math Problem Solver. Consider:\\n1. Are there any logical flaws or missing steps?\\n2. Is the proposed solution efficient and clear?\\n3. Suggest alternative methods or simplifications if applicable.\\n\\nProvide a summary of your assessment and any recommendations for refining the solution.
##### MathProblemReflector
Based on the feedback from the Math Problem Analyst, consider the following:\\n1. What specific aspects of the approach can be improved?\\n2. Are there any new angles to consider that haven't been addressed?\\n3. How can the methodology be adjusted for greater clarity and efficiency?\\n\\nProvide a revised structured approach for solving the problem.

